In [1]:
%%writefile Dockerfile

FROM python:3.9-slim-buster

RUN pip install pandas==2.2.2 scikit-learn==1.4.2 prophet==1.1.5

COPY code/preprocess.py /opt/ml/code/preprocess.py
COPY code/train.py /opt/ml/code/train.py

ENV SAGEMAKER_PROGRAM preprocess.py
ENV SAGEMAKER_PROGRAM train.py
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]

Overwriting Dockerfile


In [2]:
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.Session().region_name
ecr_repository = 'demo_prophet'
tag = ':latest'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

In [3]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository .

Sending build context to Docker daemon  159.7kB
Step 1/8 : FROM python:3.9-slim-buster
3.9-slim-buster: Pulling from library/python

b88d5577: Pulling fs layer 
16e23423: Pulling fs layer 
da260408: Pulling fs layer 
c79126f6: Pulling fs layer 
Digest: sha256:320a7a4250aba4249f458872adecf92eea88dc6abd2d76dc5c0f01cac9b53990[1A
Status: Downloaded newer image for python:3.9-slim-buster
 ---> c84dbfe3b8de
Step 2/8 : RUN pip install pandas==2.2.2 scikit-learn==1.4.2 prophet==1.1.5
 ---> Running in c8fab2d4fd8f
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 5.0 MB/s eta 0:00:00
   

In [4]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [5]:
!aws ecr create-repository --repository-name $ecr_repository


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'demo_prophet' already exists in the registry with id '654654222480'


In [6]:
!docker tag {ecr_repository + tag} $processing_repository_uri

In [7]:
!docker push $processing_repository_uri

The push refers to repository [654654222480.dkr.ecr.us-east-1.amazonaws.com/demo_prophet]

842cc9d0: Preparing 
488b5534: Preparing 
d0cba51b: Preparing 
a27560c1: Preparing 
037e08b3: Preparing 
eede8d6e: Preparing 
55769c5e: Preparing 
d0cba51b: Pushed   593.3MB/581.8MBAlatest: digest: sha256:05db1090b79cef8c739046219afd1defc1e3f9206c416c183b8f4b100a9fd52b size: 1997
